In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gykzg_gm
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gykzg_gm
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4293 sha256=60a4bb45f2e22823462287c055f0c2d8ee6c117f76fdaf10df414b7ed1b76ca0
  Stored in directory: /tmp/pip-ephem-wheel-cache-6098pohl/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%writefile code_fill.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main()
{
    int dim,i;
    FILE *fp;
    fp = fopen("cudamatrix_3000.txt","w");
    printf("Enter dimension: ");
    char buf[10];
    scanf("%d",&dim);
    sprintf(buf,"%d",dim);
    fputs(buf,fp);
    memset(buf,'\0',sizeof(buf));
    putc('\n',fp);
    int mat[dim*(dim+1)];
    for(i=0;i<dim*(dim+1);i++)
    {
        mat[i] = rand()%40;
        while(mat[i]==0)
        {
            mat[i] = rand()%40;
        }
        sprintf(buf,"%d",mat[i]);
        fputs(buf,fp);
        memset(buf,'\0',sizeof(buf));
        putc(' ',fp);
        if(i%dim==(dim-1))
            putc('\n',fp);
    }
}


Writing code_fill.c


In [6]:
!gcc -o code_fill code_fill.c

In [7]:
#!./code_fill

In [8]:
# https://github.com/abhinavmittra/GaussElimination_Parallel

%%writefile code_proc.cu

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <string.h>

#define MAXBLOCKSIZE 512

int Size;
float *a, *b, *finalVec;
float *m;
FILE *fp;

void InitProblemOnce(char *size);
void InitProblemOnceFromFile(char *filename);
void InitPerRun();
void ForwardSub();
void BackSub();
void InitMat(float *ary, int nrow, int ncol);
void InitMatFromFile(float *ary, int nrow, int ncol);
void InitAry(float *ary, int ary_size);
void InitAryFromFile(float *ary, int ary_size);
void PrintAry(float *ary, int ary_size);

float gpuTime = 0;

/*
 Calculation of Multiplier matrix to introduce zero's in each of the columns.
*/
__global__ void multiplier(float *m_cuda, float *a_cuda, int Size, int t)
{
	int idx = threadIdx.x + blockIdx.x * blockDim.x;

	if (idx >= Size - 1 - t )
  {
    return;
  }

	*(m_cuda + Size * (idx + t + 1) + t) = *(a_cuda + Size * (idx + t + 1) + t) / *(a_cuda + Size * t + t);
}

//Conversion of matrix to Upper Triangular Matrix

__global__ void upper(float *m_cuda, float *a_cuda, float *b_cuda,int Size, int j1, int t)
{
	int xidx = blockIdx.x * blockDim.x + threadIdx.x;
	int yidx = blockIdx.y * blockDim.y + threadIdx.y;

	if(xidx >= Size - 1 - t)
  {
    return;
  }
	if(yidx >= Size - t)
  {
    return;
  }

	a_cuda[Size * (xidx + 1 + t) + (yidx + t)] -= m_cuda[Size * (xidx + 1 + t) + t] * a_cuda[Size * t + (yidx + t)];

	if(yidx == 0)
  {
		b_cuda[xidx + 1 + t] -= m_cuda[Size * (xidx + 1 + t) + (yidx + t)] * b_cuda[t];
	}
}

int main(int argc, char *argv[])
{
  InitProblemOnce(argv[1]);
  //InitProblemOnceFromFile(argv[1]); //Input file
  InitPerRun(); //Initialize m with 0

  // run kernels
  ForwardSub();
  BackSub();

  //printf("The final solution is: \n");
  //PrintAry(finalVec, Size);

  printf("Time for CUDA kernels:\t%f sec\n", gpuTime / 1000.0);

  free(m);
  free(a);
  free(b);
}

/*
  Initializing all matrices and arrays
*/
void InitProblemOnce(char *size)
{
	Size = atoi(size);
  printf("Size = %d\n", Size);

	a = (float *) malloc(Size * Size * sizeof(float));
	InitMat(a, Size, Size);
	b = (float *) malloc(Size * sizeof(float));

	InitAry(b, Size);
	m = (float *) malloc(Size * Size * sizeof(float));
}

void InitProblemOnceFromFile(char *filename)
{
	fp = fopen(filename, "r");
	fscanf(fp, "%d", &Size);
	a = (float *) malloc(Size * Size * sizeof(float));
	InitMatFromFile(a, Size, Size);
	b = (float *) malloc(Size * sizeof(float));

	InitAryFromFile(b, Size);
	 m = (float *) malloc(Size * Size * sizeof(float));
}

/*------------------------------------------------------
 ** InitPerRun() -- Initialize the contents of the
 ** multipier matrix **m
 **------------------------------------------------------
 */
void InitPerRun()
{
	int i;
	for (i = 0; i < Size * Size; i++)
  {
		*(m + i) = 0.0;
  }
}

/*------------------------------------------------------
 ** ForwardSub() -- Forward substitution of Gaussian
 ** elimination.
 **------------------------------------------------------
 */
void ForwardSub()
{
	int t;
  float *m_cuda, *a_cuda, *b_cuda;

	// allocate memory on GPU
	cudaMalloc((void **) &m_cuda, Size * Size * sizeof(float));

	cudaMalloc((void **) &a_cuda, Size * Size * sizeof(float));

	cudaMalloc((void **) &b_cuda, Size * sizeof(float));

	// copy memory to GPU
	cudaMemcpy(m_cuda, m, Size * Size * sizeof(float),cudaMemcpyHostToDevice );
	cudaMemcpy(a_cuda, a, Size * Size * sizeof(float),cudaMemcpyHostToDevice );
	cudaMemcpy(b_cuda, b, Size * sizeof(float),cudaMemcpyHostToDevice );

	int block_size,  grid_size;

	block_size = MAXBLOCKSIZE;
	grid_size = (Size / block_size) + (!(Size % block_size) ? 0 : 1);

	dim3 dimBlock(block_size);
	dim3 dimGrid(grid_size);

	int blockSize2d, gridSize2d;
	blockSize2d = 256;
	gridSize2d = (Size / blockSize2d) + (!(Size % blockSize2d ? 0 : 1));

	dim3 dimBlockXY(blockSize2d, blockSize2d);
	dim3 dimGridXY(gridSize2d, gridSize2d);

  // begin timing kernels
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start, 0);

	for (t=0; t<(Size-1); t++)
  {
		multiplier<<<dimGrid, dimBlock>>>(m_cuda, a_cuda, Size, t);
		upper<<<dimGridXY, dimBlockXY>>>(m_cuda, a_cuda, b_cuda, Size, Size - t, t);
	}

  // end timing kernels
  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&gpuTime, start, stop);

	// copy memory back to CPU
	cudaMemcpy(m, m_cuda, Size * Size * sizeof(float),cudaMemcpyDeviceToHost );
	cudaMemcpy(a, a_cuda, Size * Size * sizeof(float),cudaMemcpyDeviceToHost );
	cudaMemcpy(b, b_cuda, Size * sizeof(float),cudaMemcpyDeviceToHost );
	cudaFree(m_cuda);
	cudaFree(a_cuda);
	cudaFree(b_cuda);
}

/*------------------------------------------------------
 ** BackSub() -- Backward substitution
 **------------------------------------------------------
 */

void BackSub()
{
	// create a new vector to hold the final answer
	finalVec = (float *) malloc(Size * sizeof(float));
	// solve "bottom up"
	int i, j;

	for(i = 0; i < Size; i++)
  {
		finalVec[Size - i - 1] = b[Size - i - 1];

    for(j = 0; j < i; j++)
		{
			finalVec[Size - i - 1] -= *(a + Size * (Size - i - 1) + (Size - j - 1)) * finalVec[Size - j - 1];
		}

		finalVec[Size - i - 1] = finalVec[Size - i - 1] / *(a + Size * (Size - i - 1) + (Size - i - 1));
	}
}

void InitMat(float *ary, int nrow, int ncol)
{
	int i, j;

	for(i = 0; i < nrow; i++)
  {
		for(j = 0; j < ncol; j++)
    {
      *(ary + Size * i + j) = rand()%40;
      while(*(ary + Size * i + j) == 0)
      {
          *(ary + Size * i + j) = rand()%40;
      }
		}
	}
}

void InitMatFromFile(float *ary, int nrow, int ncol)
{
	int i, j;

	printf("Initial Matrix A \n");

	for (i = 0; i < nrow; i++)
  {
		for (j = 0; j < ncol; j++)
    {
			fscanf(fp, "%f",  ary + Size * i + j);
			printf("%f ",*(ary + Size * i + j));
		}

		printf("\n");
	}
}

/*------------------------------------------------------
 ** InitAry() -- Initialize the array (vector) by reading
 ** data from the data file
 **------------------------------------------------------
 */
void InitAry(float *ary, int ary_size)
{
	int i;

  for(i = 0; i < ary_size; i++)
  {
		ary[i] = i;
	}
}

void InitAryFromFile(float *ary, int ary_size)
{
	int i;

	printf("\n Vector B is \n");

	for (i=0; i<ary_size; i++)
  {
		fscanf(fp, "%f",  &ary[i]);

		printf("%f ",ary[i]);
	}
}

/*------------------------------------------------------
 ** PrintAry() -- Print the contents of the array (vector)
 **------------------------------------------------------
 */
void PrintAry(float *ary, int ary_size)
{
	int i;
	for(i = 0; i < ary_size; i++)
  {
		printf("%.2f ", ary[i]);
	}

	printf("\n\n");
}

Writing code_proc.cu


In [9]:
!nvcc -o code_proc code_proc.cu

In [10]:
#!./code_proc cudamatrix_test.txt
!./code_proc 5000

Size = 5000
Time for CUDA kernels:	0.000000 sec
